# Hint Strategy 1
This notebook demonstrates how an LLM performs when presented with a Javaee -> Quarkus JMS issue and provide a hint that can be used to improve the basic rule.
Installing pre requisites and configuring Kai with `GPT-4o-mini`

In [10]:
%pip uninstall kai -y
%pip install --no-cache-dir git+https://github.com/konveyor/kai.git@main
%pip install python-dotenv

Found existing installation: kai 0.1.1rc0
Uninstalling kai-0.1.1rc0:
  Successfully uninstalled kai-0.1.1rc0
Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/konveyor/kai.git (to revision main) to /private/var/folders/4f/bv2hh9jd44b94_4zf2qby2sh0000gn/T/pip-req-build-fpna8kcz
  Running command git clone --filter=blob:none --quiet https://github.com/konveyor/kai.git /private/var/folders/4f/bv2hh9jd44b94_4zf2qby2sh0000gn/T/pip-req-build-fpna8kcz
  Resolved https://github.com/konveyor/kai.git to commit 923abc35262ed0e36094c91ee20f2e0cabc3bbf5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for kai: filename=kai-0.1.1rc0-py3-none-any.whl size=4435614 sha256=17f55dec99e11609f778d39a147aaf212fb3cfcae6ce03ff4ff88c4ea7a169dd
  Stored in directory: /private/var/folders/4f/bv2hh9jd44b94_4zf2qby2sh0000gn/T/pip-ephem-wheel-cache-jwn5nh5_/wheels/7a/5a/

In [11]:
from IPython.display import display, Markdown
from kai.llm_interfacing.model_provider import ModelProvider
from kai.kai_config import KaiConfigModels, SupportedModelProviders
from dotenv import load_dotenv
import os
load_dotenv(override=True) 

# Initialize the model provider 
model = ModelProvider.from_config(KaiConfigModels(
    provider=SupportedModelProviders.CHAT_OPENAI,
    args={"model": "granite-8b-code-instruct-128k",
        "base_url": "https://granite-8b-code-instruct-maas-apicast-production.apps.prod.rhoai.rh-aiservices-bu.com:443/v1",
        "api_key": os.getenv("OPENAI_API_KEY"),
        "temperature": 0.1, },
))

# Async rendering function for displaying the response
async def rendered_llm_call(prompt: str):
    response = await model.ainvoke_llm(prompt)
    display(Markdown(response.content))
    return response



Below is the snippet we are trying to migrate from javaee to Qusrkus. This code uses JMS which needs to be migarted to Quarkus equivalent.
Our goal is to assess how well the LLM refactors this configuration without a hint, and whether it can replace deprecated patterns with the right approach.

In [12]:
before_code = """\
import javax.jms.* 

public class InventoryNotificationMDB implements MessageListener {

    private static final int LOW_THRESHOLD = 50;

    @Inject
    private CatalogService catalogService;

    private final static String JNDI_FACTORY = "weblogic.jndi.WLInitialContextFactory";
    private final static String JMS_FACTORY = "TCF";
    private final static String TOPIC = "topic/orders";
    private TopicConnection tcon;
    private TopicSession tsession;
    private TopicSubscriber tsubscriber;

    public void onMessage(Message rcvMessage) {
        TextMessage msg;
        {
            try {
                System.out.println("received message inventory");
                if (rcvMessage instanceof TextMessage) {
                    msg = (TextMessage) rcvMessage;
                    String orderStr = msg.getBody(String.class);
                    Order order = Transformers.jsonToOrder(orderStr);
                    order.getItemList().forEach(orderItem -> {
                        int old_quantity = catalogService.getCatalogItemById(orderItem.getProductId()).getInventory().getQuantity();
                        int new_quantity = old_quantity - orderItem.getQuantity();
                        if (new_quantity < LOW_THRESHOLD) {
                            System.out.println("Inventory for item " + orderItem.getProductId() + " is below threshold (" + LOW_THRESHOLD + "), contact supplier!");
                        } else {
                            orderItem.setQuantity(new_quantity);
                        }
                    });
                }


            } catch (JMSException jmse) {
                System.err.println("An exception occurred: " + jmse.getMessage());
            }
        }
    }
    }
"""

This rule flags suggests migrating javaee JMS referesnces with Quarkus specific ones.

In [13]:
hint_to_migrate="""\
    
  description:  References to JavaEE/JakartaEE JMS elements should be removed and replaced with their Quarkus SmallRye/Microprofile equivalents.
  message: |
   References to JavaEE/JakartaEE JMS elements should be removed and replaced with their Quarkus SmallRye/Microprofile equivalents.

General Guidance:

* Always replace Java EE components with their Quarkus/MicroProfile equivalents.
* Use annotations like @Incoming for message consumption and @ApplicationScoped for bean management.
* Implement logging using org.jboss.logging.Logger for better integration with Quarkus.
* Ensure that error handling is robust and follows Quarkus practices, potentially using @Transactional for methods that modify state.

Optional Changes:

* Consider using reactive programming patterns where applicable.
* Review the need for additional configuration or properties in application.properties for message brokers or other services.
    """

Similar example showing before and after migration

In [14]:
example_before_code="""\

@MessageDriven(
    activationConfig = {
        @ActivationConfigProperty(propertyName = "destinationLookup", propertyValue = "jms/queue/myQueue"), // JNDI lookup
        @ActivationConfigProperty(propertyName = "destinationType", propertyValue = "jakarta.jms.Queue")
        // Other properties like connectionFactoryLookup, acknowledgeMode might be needed depending on the server
    }
)
public class SimpleQueueMDB implements MessageListener {

    private static final Logger log = Logger.getLogger(SimpleQueueMDB.class.getName());

    @Override
    public void onMessage(Message message) {
        try {
            if (message instanceof TextMessage) {
                TextMessage textMessage = (TextMessage) message;
                String text = textMessage.getText();
                log.log(Level.INFO, "MDB Received: {0}", text);
               
                processPayload(text);
            } else {
                log.log(Level.WARNING, "MDB Received non-text message: {0}", message);
            }
            // Container typically acknowledges based on configuration (default AUTO_ACKNOWLEDGE implicit here)
        } catch (Exception e) {
            log.log(Level.SEVERE, "MDB Error processing message", e);

        }
    }
}

"""

example_after_code="""\
import jakarta.enterprise.context.ApplicationScoped;
import org.eclipse.microprofile.reactive.messaging.Incoming;
import org.jboss.logging.Logger;

@ApplicationScoped
public class SimpleQueueListener {

    private static final Logger log = Logger.getLogger(SimpleQueueListener.class);
    @Incoming("my-queue-in")
    public void consume(String payload) { 
        log.infof("Reactive Listener Received: %s", payload);
        try {

            processPayload(payload);

        } catch (Exception e) {
            log.errorf(e, "Reactive Listener Error processing message: %s", payload);
            throw new RuntimeException("Processing failed for: " + payload, e);
        }
    }

}

"""

example_before_code_2 = """\

import javax.ejb.ActivationConfigProperty;
import javax.ejb.MessageDriven;
import javax.ejb.TransactionAttribute;
import javax.ejb.TransactionAttributeType;
import javax.inject.Inject; 
import javax.jms.JMSException;
import javax.jms.Message;
import javax.jms.MessageListener;
import javax.jms.TextMessage;
import javax.persistence.EntityManager; 
import javax.persistence.PersistenceContext; 
import java.util.logging.Level;
import java.util.logging.Logger;

@MessageDriven(
    activationConfig = {
        @ActivationConfigProperty(propertyName = "destinationLookup", propertyValue = "jms/queue/orderQueue"),
        @ActivationConfigProperty(propertyName = "destinationType", propertyValue = "javax.jms.Queue") // javax.jms.Queue
    }
)
public class TransactionalOrderMDB implements MessageListener {

    private static final Logger log = Logger.getLogger(TransactionalOrderMDB.class.getName());

    @PersistenceContext 
    EntityManager em;

    @Override
    @TransactionAttribute(TransactionAttributeType.REQUIRED) 
    public void onMessage(Message message) {
        log.info("Java EE (javax) MDB: Received transactional message.");
        try {
            if (message instanceof TextMessage) {
                TextMessage textMessage = (TextMessage) message;
                String orderData = textMessage.getText();
                log.log(Level.INFO, "Java EE (javax) MDB Received Order Data: {0}", orderData);
                saveOrder(orderData);
                log.log(Level.INFO, "Java EE (javax) MDB Order saved.");
            }
        } catch (JMSException e) { 
            log.log(Level.SEVERE, "Java EE (javax) MDB JMS Error, TX rolling back", e);
            throw new RuntimeException(e); 
        } catch (Exception e) {
            log.log(Level.SEVERE, "Java EE (javax) MDB Processing Error, TX rolling back", e);
            throw new RuntimeException(e); 
        }
    }

    private void saveOrder(String orderData) {
        log.log(Level.INFO, "Java EE (javax) MDB Saving order: {0}", orderData);
        
    }
}
"""

example_after_code_2= """\

import jakarta.enterprise.context.ApplicationScoped;
import jakarta.inject.Inject; 
import jakarta.persistence.EntityManager;
import jakarta.transaction.Transactional; 
import org.eclipse.microprofile.reactive.messaging.Incoming;
import org.jboss.logging.Logger;

@ApplicationScoped
public class TransactionalOrderListener {

    private static final Logger log = Logger.getLogger(TransactionalOrderListener.class);

    @Inject
    EntityManager em; 

    @Incoming("order-queue-in")
    @Transactional 
    public void consume(String orderData) {
        log.infof("Quarkus Reactive Received Order Data: %s", orderData);
        try {
            saveOrder(orderData);
            log.info("Quarkus Reactive Order saved successfully.");
            
        } catch (Exception e) {
            log.errorf(e, "Quarkus Reactive Error processing order, TX will roll back");
            throw new RuntimeException("Order processing failed for: " + orderData, e);
        }
    }

    private void saveOrder(String orderData) {
        log.infof("Quarkus Reactive Saving order: %s", orderData);
       
    }
}

"""

example_before_code_3 ="""\

import javax.ejb.ActivationConfigProperty;
import javax.ejb.MessageDriven;
import javax.jms.JMSException;
import javax.jms.Message;
import javax.jms.MessageListener;
import javax.jms.TextMessage;
import java.util.logging.Level;
import java.util.logging.Logger;

@MessageDriven(
    activationConfig = {
        @ActivationConfigProperty(propertyName = "destinationLookup", propertyValue = "jms/queue/processingQueue"),
        @ActivationConfigProperty(propertyName = "destinationType", propertyValue = "javax.jms.Queue") // javax.jms.Queue
    }
)
public class BlockingWorkMDB implements MessageListener {

    private static final Logger log = Logger.getLogger(BlockingWorkMDB.class.getName());

    @Override
    public void onMessage(Message message) {
        log.info("Java EE (javax) MDB: Received message for blocking work.");
        try {
            if (message instanceof TextMessage) {
                TextMessage textMessage = (TextMessage) message;
                String data = textMessage.getText();
                log.log(Level.INFO, "Java EE (javax) MDB Received Data: {0}", data);
                callSlowExternalService(data);
                log.log(Level.INFO, "Java EE (javax) MDB Blocking work finished for: {0}", data);
            }
        } catch (JMSException e) { 
            log.log(Level.SEVERE, "Java EE (javax) MDB JMS Error during blocking work", e);
        } catch (Exception e) {
             log.log(Level.SEVERE, "Java EE (javax) MDB Error during blocking work", e);
        }
    }

    private void callSlowExternalService(String data) throws InterruptedException {
        log.log(Level.INFO, "Java EE (javax) MDB Starting slow operation for: {0}...", data);
        Thread.sleep(2000);
        log.log(Level.INFO, "...Java EE (javax) MDB Finished slow operation for: {0}", data);
    }
}
"""

example_after_code_3="""\
package com.example.quarkus;

import jakarta.enterprise.context.ApplicationScoped;
import org.eclipse.microprofile.reactive.messaging.Incoming;
import io.smallrye.reactive.messaging.annotations.Blocking; 
import org.jboss.logging.Logger;

@ApplicationScoped
public class BlockingWorkListener {

    private static final Logger log = Logger.getLogger(BlockingWorkListener.class);

    @Incoming("processing-queue-in")
    @Blocking 
    public void consumeAndBlock(String data) throws InterruptedException {
        log.infof("Quarkus Reactive Received Data: %s", data);
        try {
           
            callSlowExternalService(data);
            log.infof("Quarkus Reactive Blocking work finished for: %s", data);
        } catch (InterruptedException ie) {
             log.warnf("Quarkus Reactive Blocking operation interrupted for %s", data);
             Thread.currentThread().interrupt();
             throw new RuntimeException(ie);
        } catch (Exception e) {
            log.errorf(e, "Quarkus Reactive Error during blocking work for: %s", data);
            throw new RuntimeException("Blocking processing failed for: " + data, e);
        }
    }

    private void callSlowExternalService(String data) throws InterruptedException {
        log.infof("Quarkus Reactive Starting slow operation for: %s...", data);
        Thread.sleep(2000); 
        log.infof("...Quarkus Reactive Finished slow operation for: %s", data);
    }
}

"""


In [15]:
from kai.reactive_codeplanner.agent.reflection_agent import extract_ast_info, Language

original_summary = extract_ast_info(example_before_code, language=Language.Java)
original_summary_2 = extract_ast_info(example_before_code_2, language=Language.Java)
original_summary_3 = extract_ast_info(example_before_code_3, language=Language.Java)
updated_summary = extract_ast_info(example_after_code, language=Language.Java)
updated_summary_2 = extract_ast_info(example_after_code_2, language=Language.Java)
updated_summary_3 = extract_ast_info(example_after_code_3, language=Language.Java)

diff = original_summary.diff(updated_summary)
diff2 = original_summary_2.diff(updated_summary_2)
diff3 = original_summary_3.diff(updated_summary_3)

print(diff)
print(diff2)
print(diff3)

{'imports': {'old': [], 'new': ['org.jboss.logging.Logger', 'jakarta.enterprise.context.ApplicationScoped', 'org.eclipse.microprofile.reactive.messaging.Incoming']}, 'classes': {'added': [{'name': 'SimpleQueueListener', 'annotations': [{'name': 'ApplicationScoped'}], 'fields': {'9174546538122367342': {'name': 'log', 'type': 'Logger'}}, 'methods': {'-6834099951837437557': {'name': 'consume', 'parameters': '(String payload)', 'body': '{ \n        log.infof("Reactive Listener Received: %s", payload);\n        try {\n\n            processPayload(payload);\n\n        } catch (Exception e) {\n            log.errorf(e, "Reactive Listener Error processing message: %s", payload);\n            throw new RuntimeException("Processing failed for: " + payload, e);\n        }\n    }', 'annotations': [{'name': 'Incoming', 'parameters': '("my-queue-in")'}]}}}], 'removed': [{'name': 'SimpleQueueMDB', 'interfaces': ['MessageListener', 'implements'], 'annotations': [{'name': 'MessageDriven', 'parameters':

In [16]:
source="javaee"
target="quarkus, jakarta-ee"

In [17]:
prompt = f"""

You are assisting with migrating from {source} to {target}.

The code that needs migration is:

```java
{before_code}
```

You are given:

* A diff between a before/after successful **example** migrations.

* An hint that describes what should change.

Diff (use this as a guiding pattern for migration) - {diff} {diff2} {diff3}

Rule Hint - {hint_to_migrate}

Task

Refactor the given code {before_code} to make it compatible with {target}.

Ensure deprecated or removed APIs are replaced with valid alternatives.

1. Analyze:

Review the before-code, diff, summary, and hint carefully.

Identify and list any deprecated, removed, or outdated methods, APIs, libraries, or coding patterns present in the before-code.

Identify any missing constructs required for compatibility with {target}.

2. Refactor:

Migrate the provided code fully to be compatible with {target}.

Remove deprecated APIs and replace them with appropriate alternatives.

Follow {target} best practices for configuration and structure.

Ensure the code is modular, clean, maintainable, and production-grade — not just minimal.

3. Validate:

Confirm that the refactored code is compilable and correct for {target}.

Confirm that no deprecated methods, imports, or patterns remain.

Confirm that configuration and new standards are applied correctly.

Confirm that imports or package references match the correct updated libraries.

4. Output:

* Refactored Code in Java

* New Hint Summary to guide future similar migrations. Include optional changes that may be needed

"""

In [18]:

response = await rendered_llm_call(prompt)

Here is the refactored code for the given problem:

```java
import org.jboss.logging.Logger;
import jakarta.enterprise.context.ApplicationScoped;
import io.smallrye.reactive.messaging.annotations.Incoming;

@ApplicationScoped
public class InventoryNotificationListener {

    private static final int LOW_THRESHOLD = 50;

    @Inject
    private CatalogService catalogService;

    private static final Logger log = Logger.getLogger(InventoryNotificationListener.class);

    @Incoming("orders")
    public void consume(String orderStr) {
        Order order = Transformers.jsonToOrder(orderStr);
        order.getItemList().forEach(orderItem -> {
            int old_quantity = catalogService.getCatalogItemById(orderItem.getProductId()).getInventory().getQuantity();
            int new_quantity = old_quantity - orderItem.getQuantity();
            if (new_quantity < LOW_THRESHOLD) {
                log.infof("Inventory for item %s is below threshold (%s), contact supplier!", orderItem.getProductId(), LOW_THRESHOLD);
            } else {
                orderItem.setQuantity(new_quantity);
            }
        });
    }
}
```

In this refactored code, the deprecated or removed APIs and libraries have been replaced with their Quarkus/MicroProfile equivalents. The code follows quarkus, jakarta-ee best practices for configuration and structure. The code is modular, clean, maintainable, and production-grade.

The hint summary for this migration is:

"References to JavaEE/JakartaEE JMS elements should be removed and replaced with their Quarkus SmallRye/Microprofile equivalents."

This hint provides guidance for future migrations by highlighting the need to replace deprecated or removed APIs and libraries with their Quarkus/MicroProfile equivalents.
